In [1]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("movies.csv", encoding='latin-1')
data.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [5]:
#Data understanding
for column in data.columns:
    print(data[column].value_counts())

0.0            2182
20000000.0      207
30000000.0      191
25000000.0      177
15000000.0      170
10000000.0      155
40000000.0      150
35000000.0      149
50000000.0      115
5000000.0       113
60000000.0      107
12000000.0      102
6000000.0        90
8000000.0        88
18000000.0       88
13000000.0       76
7000000.0        73
3000000.0        73
45000000.0       72
17000000.0       65
80000000.0       65
70000000.0       65
11000000.0       62
16000000.0       60
14000000.0       58
22000000.0       57
4000000.0        55
75000000.0       54
55000000.0       54
100000000.0      49
               ... 
260000000.0       1
8900000.0         1
12800000.0        1
10920000.0        1
63600000.0        1
9200000.0         1
65000.0           1
126000000.0       1
29750000.0        1
17700000.0        1
176000000.0       1
169000.0          1
26350000.0        1
5600000.0         1
230000000.0       1
144000000.0       1
11800000.0        1
950000.0          1
2309723.0         1


In [29]:
#Get the names of movies with no budget and format the string for a URL
noBudgetMovies = data[data["budget"]==0][["year","name"]]
noBudgetMovies["name"] = noBudgetMovies["name"].str.replace(" ","_")
noBudgetMovies.head()

,year,name
25,1986,Short_Circuit
26,1986,The_Name_of_the_Rose
27,1986,Iron_Eagle
32,1986,Betty_Blue
35,1986,The_Karate_Kid_Part_II


In [ ]:
from bs4 import BeautifulSoup
import requests
import time

def getPage(movieName):
    S = requests.Session()
    URL = "https://en.wikipedia.org/w/api.php"
    PARAMS = {
        "action": "parse",
        "page": movieName,
        "format": "json"
    }
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA

#Getting the budget value
def getBudget(DATA):
    try:
        soup = BeautifulSoup("<html>"+str(DATA["parse"]["text"]["*"])+"</html>",features="html.parser")
    except:
        return "parseError"
    table = soup.find("table",class_="infobox vevent")

    try:
        thList = table.find_all("th")
        tdList = table.find_all("td")
    except:
        return "redirect"

    budgetIndex = -1
    i=0
    for row in thList:
        if "Budget" in str(row):
            budgetIndex = i
            break
        i += 1

    if budgetIndex == -1:
        return "noBudget"
    
    return tdList[budgetIndex].contents[0]


errors = []
budgets = []

#errors2 = []
#budgets2 = []

#print(getBudget(getPage("The_Karate_Kid_Part_II")))

#print(getPage("The_Name_of_the_Rose")["parse"]["text"]["*"])

for movieName in noBudgetMovies["name"]:
    #print(movieName)
    pageData = getPage(movieName)
    #print(pageData["parse"]["text"]["*"])
    #time.sleep(5)
    if pageData=="redirect":
        errors.append("Table was NoneType. probably wrong page. Movie: "+movieName)
        continue
    if pageData=="noBudget":
        errors.append("No budget found. Movie: "+movieName)
        continue
    if pageData=="parseError":
        errors.append("No page found. Movie: "+movieName)
        continue
#     if len(pageData["parse"]["text"]["*"]) < 10000:
#         errors.append("Length is under 10000. Movie: "+movieName)
#         continue
    budget = getBudget(pageData)
    budgets.append(budget)

In [ ]:
print(errors)
print(budgets)